In [27]:
import numpy as np
import re
import string
import os
import hashlib
import sortedcontainers
from sortedcontainers import SortedDict
import pickle
import time
lst = ['VAR','LP','RP','AND','OR','NLP']

## Expression Tree for our request

In [51]:
class TokenType(object):
    VAR,LP,RP,AND,OR,NLP = range(6)

class TreeNode(object):
    def __init__(self,tokenType):
        self.tokenType = tokenType
        self.value = None
        self.left = None
        self.right = None

class Tokenizer(object):
    def __init__(self,exp):
        self.expression = exp
        self.tokens = None
        self.tokenTypes = None
        self.i = 0
        
    def next(self):
        self.i += 1
        return self.tokens[self.i - 1]
    
    def peek(self):
        return self.tokens[self.i]
    
    def hasNext(self):
        return self.i < len(self.tokens)
    
    def nextTokenType(self):
        return self.tokenTypes[self.i]
    
    def tokenize(self):
        regex = re.compile(r'(\bAND\b|\bOR\b|\(|!\(|\))')
        self.tokens = regex.split(self.expression)
        self.tokens = [t.strip() for t in self.tokens if t.strip() != '']
        if self.tokens[0] == self.expression:
            self.tokens = self.expression.split()
        print('Tokens:',self.tokens)
        self.tokenTypes = []
        idx = 0
        for t in self.tokens:
            if t == 'AND':
                self.tokenTypes.append(TokenType.AND)
            elif t == 'OR':
                self.tokenTypes.append(TokenType.OR)
            elif t == '(':
                self.tokenTypes.append(TokenType.LP)
            elif t == '!(':
                self.tokenTypes.append(TokenType.NLP)
            elif t == ')':
                self.tokenTypes.append(TokenType.RP)
            else:
                self.tokenTypes.append(TokenType.VAR)
                idx+=1
        if idx == len(self.tokens):
            for jdx in range(1,len(self.expression),2):
                self.tokenTypes.insert(jdx,TokenType.AND)
        print('tokenTypes:', self.tokenTypes)

class BooleanParser(object):
    def __init__(self,exp,log_file):
        self.exp = exp
        self.log_file = log_file
        self.tokenizer = Tokenizer(exp)
        self.tokenizer.tokenize()
        with open(self.log_file,'w') as stream:
            stream.write('Parsed expression info:\nOperands: {0} \nOperations: {1}\n'.format(str(self.tokenizer.tokens),str([lst[idx] for idx in self.tokenizer.tokenTypes])))
        self.parse()
        
    def parse(self):
        with open(self.log_file,'a') as stream:
            stream.write('Parsing expression: ' + self.exp + '\n')
            self.root = self.parseExpression(stream)
    
    def parseExpression(self,stream):
        stream.write('Parsing expression.' + 'Format expression: AndClause {OR AndClause}.\nBuilding Expression Tree......\n')
        andTerm1 = self.parseAndTerm(stream)
        stream.write("Parsing expression: Token Type for term 1: " + str(lst[andTerm1.tokenType]) + '\n')
        while self.tokenizer.hasNext() and self.tokenizer.nextTokenType() == TokenType.OR:
            self.tokenizer.next()
            andTerm2 = self.parseAndTerm(stream)
            stream.write("Parse expression: Token Type for term 2: " + str(lst[andTerm2.tokenType]) + '\n')
            andTerm = TreeNode(TokenType.OR)
            andTerm.left = andTerm1
            andTerm.right = andTerm2
            andTerm1 = andTerm
        stream.write('End Parsing.......\nComplete. Root operation: {0}\n'.format(str(lst[andTerm1.tokenType])))
        return andTerm1
        
    def parseAndTerm(self,stream):
        stream.write('Parsing AndClause: [Operand {AND Operand}+ ]\n')
        operand1 = self.parseOperand(stream)
        stream.write("Parsing expression: Token Type for operand 1: " + str(lst[operand1.tokenType]) + '\n')
        while self.tokenizer.hasNext() and self.tokenizer.nextTokenType() == TokenType.AND:
            self.tokenizer.next()
            operand2 = self.parseOperand(stream)
            stream.write("Parse expression: Token Type for operand 2: " + str(lst[operand2.tokenType]) + '\n')
            operand = TreeNode(TokenType.AND)
            operand.left = operand1
            operand.right = operand2
            operand1 = operand
            stream.write('End Parsing.......\nComplete. Node operation: {0}\n'.format(str(lst[operand1.tokenType])))
        return operand1
    
    def parseOperand(self,stream):
        stream.write('Parsing Operand: [Terminal | (Expression)] \n')
        if self.tokenizer.hasNext() and self.tokenizer.nextTokenType() == TokenType.LP:
            stream.write('Parsing Operand Expression\n')
            self.tokenizer.next()
            expression = self.parseExpression(stream)
            if self.tokenizer.hasNext() and self.tokenizer.nextTokenType() == TokenType.RP:
                self.tokenizer.next()
                return expression
            else:
                raise Exception("Closing ) expected, but got " + self.tokenizer.next())
        
        terminal =  self.parseTerminal(stream)
        return terminal
    
    def parseTerminal(self,stream):
        stream.write('Parsing variable:\n')
        if self.tokenizer.hasNext():
            tokenType = self.tokenizer.nextTokenType()
            var = TreeNode(tokenType)
            var.value = self.tokenizer.next()
            stream.write('Type of Variable: {0},Value: {1}\n'.format(lst[tokenType],var.value))
            return var
        else:
            raise Exception('NUM, STR, or VAR expected, but got ' + self.tokenizer.next())
    
    def evaluate(self,variable_dict):
        with open(self.log_file,'a') as stream:
            stream.write('Evaluating Expression: {0}\n'.format(self.exp))
            answer = self.evaluateRecursive(self.root,variable_dict,stream)
            stream.write('Answer Expression: {0}\n'.format(answer))
        return answer
    
    def evaluateRecursive(self,node,variable_dict,stream):
        stream.write('Node Type: {0}\n'.format(lst[node.tokenType]))
        if node.tokenType == TokenType.VAR:
            #print('Variable Clause:\n Name: {0} Value: {1}\n'.format(node.value,variable_dict.get(node.value)))
            stream.write('Variable Clause:\n Name: {0} Value: {1}\n'.format(node.value,variable_dict.get(node.value)))
            return variable_dict.get(node.value)
        left = self.evaluateRecursive(node.left,variable_dict,stream)
        right = self.evaluateRecursive(node.right,variable_dict,stream)
        #print('Expression Operands:\n Left Operand: {0}, Right Operand: {1}\n'.format(left,right))
        if node.tokenType == TokenType.AND:
            print('AND Result: {0}\n'.format(len(left.intersection(right))))
            stream.write('AND Result: {0}\n'.format(left.intersection(right)))
            return left.intersection(right)
        elif node.tokenType == TokenType.OR:
            print('OR Result: {0}\n'.format(len(left.union(right))))
            stream.write('OR Result: {0}\n'.format(left.union(right)))
            return left.union(right)
        else:
            raise Exception('Unexpected type ' + str(node.tokenType))



        
            
            
        

### Test Expression Tree

In [52]:
p = BooleanParser('((a OR b) AND b ) AND c','log.txt')
p.evaluate(variable_dict = {'a':set([1,2,3]),'b':set([2,3,4]),'c':set([5,6,7])})

Tokens: ['(', '(', 'a', 'OR', 'b', ')', 'AND', 'b', ')', 'AND', 'c']
tokenTypes: [1, 1, 0, 4, 0, 2, 3, 0, 2, 3, 0]
OR Result: 4

AND Result: 3

AND Result: 0



set()

## Preprocessing and evaluate expression
## Мое дерево выражений не слишком умеет работать с отрицанием целого выражения, поэтому требуется предобработать поисковый запрос до посылки в дерево (случаи пробклов -> AND, !выражения -> !операнд1 !оператор !операнд2)

In [53]:
class SearchEngine(object):
    def __init__(self,index:dict,log_file,represent = 1):
        self.represent = represent
        self.log_file = log_file
        with open(self.log_file,'w') as main_stream:
            main_stream.write('Loading Boolean Index.....\n')
            if represent == 1:
                main_stream.write('Selected tree mode. Loading tree.......\n')
                self.tree_dict = SortedDict()
                with open(index['tree_path'],'rb') as stream:
                    self.tree_dict = pickle.load(stream)
                main_stream.write('Tree loaded.\n')
            elif  represent == 2:
                main_stream.write('Selected dict mode. Loading dict.......\n')
                with open(index['word_dictionary'],'rb') as stream:
                    self.dict = pickle.load(stream)
                main_stream.write('Dict Loaded.\n')
                main_stream.write('Coordinates blocks loading....\n')
                with open(index['coo_blocks'],'rb') as stream:
                    self.coord_blocks = pickle.load(stream)
                main_stream.write('Coordinates blocks loaded.\n')
                main_stream.write('Complete.\n')
            else:
                raise NotImplementedError('no such representation')
            
    def __string2hash(self,text):
        return int(hashlib.sha256(text.encode("utf-8")).hexdigest()[:16], 16)
    
    def Preprocessing(self,request):
        main_stream = open(self.log_file,'a')
        main_stream.write('Starting Preprocess request: {0}\n'.format(request))
        dictionary = {}
        main_stream.write('Fill empties by AND\n')
        request = request.lower().strip()
        main_stream.write('Replace classical operators || and && by their alternatives: OR AND\n')
        request = request.replace('||','OR').replace('&&','AND') 
        bad_tokens = request.split()
        occurence = [i for i,x in enumerate(bad_tokens) if x == 'AND' or x == 'OR']
        main_stream.write('Operators occurence positions: {0}\n'.format(occurence))
        pos = []
        print(bad_tokens)
        main_stream.write('Bad split algo: {0}\n'.format(bad_tokens))
        if len(bad_tokens) - len(occurence) > 1:
            idx = 0
            while idx <= len(bad_tokens) - 2:
                print(idx)
                print(bad_tokens[idx])
                if bad_tokens[idx + 1] not in ('AND','OR'):
                    pos.append(idx + 1)
                    idx+=1
                else:
                    idx+=2
                    print(bad_tokens[idx])
            r = 0
            main_stream.write('Position to insert AND operator: {0}\n'.format(pos))
            for p in pos:
                bad_tokens.insert(p + r,'AND')
                r+=1
            request = ' '.join(bad_tokens)
            main_stream.write('Temp preprocessed request:{0}\n'.format(request))
        print(pos)
        # first find all !( operator
        main_stream.write('Negate expression.....\n')
        regex = re.compile(r'(\bAND\b|\bOR\b|\(|!\(|\))')
        tokens = regex.split(request)
        main_stream.write('Tokens(splitted by operations): {0}\n'.format(tokens))
        occurence = [i for i,x in enumerate(tokens) if x == '!(']
        main_stream.write("!( occurence index: {0}\n".format(occurence))
        print(occurence)
        print(tokens)
        for pos in occurence:
            print('here')
            idx = 0
            opened = 0
            while True:
                if idx == 0:
                    tokens[pos + idx] = '('
                elif tokens[pos + idx] == '(':
                    opened +=1
                elif tokens[pos + idx] == ')':
                    if opened == 0:
                        break
                    opened -=1
                elif tokens[pos + idx] == 'OR':
                        tokens[pos + idx] = 'AND'
                elif tokens[pos + idx] == 'AND':
                    tokens[pos + idx] = 'OR'
                    
                else:
                    if tokens[pos + idx].strip() != '':
                        tokens[pos + idx] = "!" + tokens[pos + idx].strip()
                idx+=1
        main_stream.write("Resulting request: {0}\n".format(" ".join(tokens)))
        main_stream.close()
        return " ".join(tokens)
    
    def SearchRequest(self,request):
        main_stream = open(self.log_file,'a')
        main_stream.write('Search request.......\n')
        rdict = {}
        request = self.Preprocessing(request)
        regex = re.compile(r'(\bAND\b|\bOR\b|\(|!\(|\))')
        tokens = regex.split(request)
        tokens = list(map(lambda x:x.strip(),tokens))
        print('Tokens:',tokens)
        main_stream.write('Tokens of request: {0}\n'.format(tokens))
        main_stream.write('Building variable dict......\n')
        for token in tokens:
            if token not in ('AND','OR','(',')'):
                if token.strip()!= "":
                    if token.strip()[0] == '!':
                        lst = list(range(40000))
                        if self.represent == 1:
                            print('rwith!',token[1:])
                            main_stream.write('String: {0}\n'.format(token))
                            rlst = self.tree_dict[self.__string2hash(token[1:])]
                            rdict[token] = set(lst).difference(set(rlst))
                            main_stream.write('String length: {0}. Value: {1}\n'.format(len(rdict),rdict))
                            print(len(rdict))
                        else:
                            rlst = self.coord_blocks[self.dict.index(token[1:])]
                            rdict[token] = set(lst).difference(set(rlst))
                            print(rdict)
                    else:
                        if self.represent == 1:
                            print('rwithout!',token)
                            main_stream.write('String: {0}\n'.format(token))
                            rlst = self.tree_dict[self.__string2hash(token.strip())]
                            rdict[token] = set(rlst)
                            main_stream.write('String length: {0}. Value: {1}\n'.format(len(rdict),rdict))
                            print(len(rdict))
                        else:
                            rlst = self.coord_blocks[self.dict.index(token)]
                            rdict[token.strip()] = set(rlst)
        main_stream.write('Evaluating...\n')
        res = BooleanParser(request,'calculus_' + self.log_file)
        res = res.evaluate(variable_dict=rdict)
        for item in res:
            main_stream.write('doc_id: {0}\n'.format(item))
        main_stream.close()
        return res
        
                
        

In [54]:
r = SearchEngine(index = {'tree_path':'table.pickle'},log_file = 'log.txt',represent = 1)

## Тест кейсы

## Пересечение часто встречающихся токенов с увеличением объема

In [55]:
%time r.SearchRequest('или из (однако || род)')

['или', 'из', '(однако', 'OR', 'род)']
0
или
1
из
2
(однако
род)
[1, 2]
[]
['или ', 'AND', ' из ', 'AND', ' ', '(', 'однако ', 'OR', ' род', ')', '']
Tokens: ['или', 'AND', 'из', 'AND', '', '(', 'однако', 'OR', 'род', ')', '']
rwithout! или
1
rwithout! из
2
rwithout! однако
3
rwithout! род
4
Tokens: ['или', 'AND', 'из', 'AND', '(', 'однако', 'OR', 'род', ')']
tokenTypes: [0, 3, 0, 3, 1, 0, 4, 0, 2]
AND Result: 16103

OR Result: 16107

AND Result: 9612

Wall time: 147 ms


{0,
 1,
 2,
 3,
 32771,
 5,
 6,
 32773,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 32783,
 32789,
 25,
 26,
 32788,
 28,
 29,
 31,
 32,
 32802,
 36,
 32804,
 38,
 39,
 32807,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 32822,
 55,
 56,
 57,
 58,
 32825,
 60,
 61,
 63,
 65,
 32833,
 32835,
 32836,
 69,
 70,
 71,
 72,
 32838,
 32839,
 75,
 76,
 77,
 78,
 32840,
 81,
 83,
 84,
 85,
 86,
 87,
 92,
 32861,
 32862,
 96,
 32864,
 32866,
 32867,
 100,
 101,
 103,
 32871,
 105,
 106,
 107,
 109,
 114,
 115,
 117,
 119,
 120,
 121,
 123,
 125,
 126,
 127,
 32894,
 129,
 130,
 131,
 32898,
 133,
 32901,
 32902,
 32903,
 32899,
 32900,
 32910,
 32911,
 144,
 32912,
 146,
 147,
 148,
 150,
 151,
 152,
 153,
 154,
 155,
 32918,
 157,
 32925,
 32926,
 163,
 32931,
 165,
 166,
 167,
 32934,
 32937,
 174,
 175,
 176,
 32946,
 179,
 32947,
 181,
 32951,
 32954,
 187,
 190,
 191,
 193,
 32964,
 32968,
 202,
 32970,
 32974,
 206,
 207,
 208,
 209,
 210,

## Текст запроса находится строго в одном документе (документ 3 про мамонтов)

In [56]:
%time r.SearchRequest('ма монты вымерший род млекопитающих живших')

['ма', 'монты', 'вымерший', 'род', 'млекопитающих', 'живших']
0
ма
1
монты
2
вымерший
3
род
4
млекопитающих
[1, 2, 3, 4, 5]
[]
['ма ', 'AND', ' монты ', 'AND', ' вымерший ', 'AND', ' род ', 'AND', ' млекопитающих ', 'AND', ' живших']
Tokens: ['ма', 'AND', 'монты', 'AND', 'вымерший', 'AND', 'род', 'AND', 'млекопитающих', 'AND', 'живших']
rwithout! ма
1
rwithout! монты
2
rwithout! вымерший
3
rwithout! род
4
rwithout! млекопитающих
5
rwithout! живших
6
Tokens: ['ма', 'AND', 'монты', 'AND', 'вымерший', 'AND', 'род', 'AND', 'млекопитающих', 'AND', 'живших']
tokenTypes: [0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0]
AND Result: 1

AND Result: 1

AND Result: 1

AND Result: 1

AND Result: 1

Wall time: 14 ms


{3}

In [ ]:
## Данный запрос был взят из документа wiki11.txt (про государственность), при проверке полученных идентификаторов документов были обнаружены данные термы

In [58]:
%time r.SearchRequest('(всю && (жизнь служить)) государству')

['(всю', 'AND', '(жизнь', 'служить))', 'государству']
0
(всю
(жизнь
2
(жизнь
3
служить))
[3, 4]
[]
['', '(', 'всю ', 'AND', ' ', '(', 'жизнь ', 'AND', ' служить', ')', '', ')', ' ', 'AND', ' государству']
Tokens: ['', '(', 'всю', 'AND', '', '(', 'жизнь', 'AND', 'служить', ')', '', ')', '', 'AND', 'государству']
rwithout! всю
1
rwithout! жизнь
2
rwithout! служить
3
rwithout! государству
4
Tokens: ['(', 'всю', 'AND', '(', 'жизнь', 'AND', 'служить', ')', ')', 'AND', 'государству']
tokenTypes: [1, 0, 3, 1, 0, 3, 0, 2, 2, 3, 0]
AND Result: 451

AND Result: 230

AND Result: 36

Wall time: 16.1 ms


{11,
 25,
 643,
 803,
 1347,
 1388,
 1400,
 1438,
 1443,
 1896,
 3908,
 5814,
 6055,
 6903,
 7920,
 8445,
 9367,
 9971,
 10326,
 11180,
 11248,
 12673,
 14459,
 14793,
 17725,
 18737,
 19180,
 19187,
 21932,
 26554,
 27953,
 28113,
 28717,
 29057,
 33535,
 34733}

In [59]:
%time r.SearchRequest('правительством && (феврале || (года (вступил || силу)))')

['правительством', 'AND', '(феврале', 'OR', '(года', '(вступил', 'OR', 'силу)))']
0
правительством
(феврале
2
(феврале
(года
4
(года
5
(вступил
силу)))
[5]
[]
['правительством ', 'AND', ' ', '(', 'феврале ', 'OR', ' ', '(', 'года ', 'AND', ' ', '(', 'вступил ', 'OR', ' силу', ')', '', ')', '', ')', '']
Tokens: ['правительством', 'AND', '', '(', 'феврале', 'OR', '', '(', 'года', 'AND', '', '(', 'вступил', 'OR', 'силу', ')', '', ')', '', ')', '']
rwithout! правительством
1
rwithout! феврале
2
rwithout! года
3
rwithout! вступил
4
rwithout! силу
5
Tokens: ['правительством', 'AND', '(', 'феврале', 'OR', '(', 'года', 'AND', '(', 'вступил', 'OR', 'силу', ')', ')', ')']
tokenTypes: [0, 3, 1, 0, 4, 1, 0, 3, 1, 0, 4, 0, 2, 2, 2]
OR Result: 4131

AND Result: 3357

OR Result: 4728

AND Result: 616

Wall time: 37.2 ms


{0,
 1,
 11,
 13,
 14,
 15,
 16,
 18,
 19,
 21,
 22,
 28,
 39,
 41,
 46,
 50,
 52,
 55,
 56,
 60,
 61,
 81,
 150,
 295,
 299,
 301,
 338,
 356,
 385,
 397,
 399,
 401,
 407,
 434,
 477,
 556,
 561,
 570,
 571,
 572,
 573,
 581,
 606,
 607,
 611,
 620,
 622,
 623,
 765,
 769,
 772,
 774,
 777,
 781,
 803,
 845,
 850,
 853,
 854,
 855,
 859,
 861,
 864,
 870,
 874,
 899,
 944,
 965,
 989,
 990,
 994,
 999,
 1008,
 1025,
 1053,
 1104,
 1106,
 1145,
 1147,
 1150,
 1151,
 1160,
 1161,
 1163,
 1166,
 1167,
 1168,
 1171,
 1193,
 1198,
 1209,
 1212,
 1233,
 1240,
 1243,
 1244,
 1245,
 1246,
 1247,
 1250,
 1253,
 1259,
 1261,
 1262,
 1267,
 1268,
 1269,
 1271,
 1284,
 1291,
 1296,
 1297,
 1300,
 1308,
 1312,
 1314,
 1322,
 1323,
 1348,
 1353,
 1373,
 1382,
 1388,
 1393,
 1418,
 1429,
 1439,
 1597,
 1639,
 1673,
 1725,
 1728,
 1859,
 1886,
 1896,
 1932,
 1960,
 2041,
 2170,
 2401,
 2419,
 2424,
 2446,
 2498,
 2517,
 2604,
 2614,
 2686,
 2805,
 3162,
 3242,
 3866,
 3959,
 3963,
 4122,
 4261,
 443

In [60]:
%time r.SearchRequest('(заодно || или) && правительством && (феврале || (года (вступил || силу)))')

['(заодно', 'OR', 'или)', 'AND', 'правительством', 'AND', '(феврале', 'OR', '(года', '(вступил', 'OR', 'силу)))']
0
(заодно
или)
2
или)
правительством
4
правительством
(феврале
6
(феврале
(года
8
(года
9
(вступил
силу)))
[9]
[]
['', '(', 'заодно ', 'OR', ' или', ')', ' ', 'AND', ' правительством ', 'AND', ' ', '(', 'феврале ', 'OR', ' ', '(', 'года ', 'AND', ' ', '(', 'вступил ', 'OR', ' силу', ')', '', ')', '', ')', '']
Tokens: ['', '(', 'заодно', 'OR', 'или', ')', '', 'AND', 'правительством', 'AND', '', '(', 'феврале', 'OR', '', '(', 'года', 'AND', '', '(', 'вступил', 'OR', 'силу', ')', '', ')', '', ')', '']
rwithout! заодно
1
rwithout! или
2
rwithout! правительством
3
rwithout! феврале
4
rwithout! года
5
rwithout! вступил
6
rwithout! силу
7
Tokens: ['(', 'заодно', 'OR', 'или', ')', 'AND', 'правительством', 'AND', '(', 'феврале', 'OR', '(', 'года', 'AND', '(', 'вступил', 'OR', 'силу', ')', ')', ')']
tokenTypes: [1, 0, 4, 0, 2, 3, 0, 3, 1, 0, 4, 1, 0, 3, 1, 0, 4, 0, 2, 2, 2]
OR Result

{0,
 1,
 11,
 13,
 14,
 15,
 16,
 18,
 19,
 21,
 22,
 28,
 39,
 41,
 46,
 50,
 52,
 55,
 56,
 60,
 61,
 81,
 150,
 295,
 299,
 301,
 338,
 356,
 385,
 397,
 399,
 401,
 407,
 477,
 561,
 570,
 571,
 581,
 606,
 611,
 620,
 622,
 623,
 772,
 774,
 777,
 781,
 803,
 845,
 850,
 853,
 854,
 855,
 859,
 861,
 864,
 870,
 874,
 899,
 944,
 965,
 989,
 990,
 994,
 999,
 1008,
 1025,
 1053,
 1104,
 1151,
 1160,
 1166,
 1193,
 1198,
 1212,
 1233,
 1240,
 1243,
 1244,
 1245,
 1246,
 1247,
 1250,
 1253,
 1259,
 1261,
 1268,
 1269,
 1271,
 1284,
 1291,
 1296,
 1297,
 1300,
 1308,
 1312,
 1314,
 1322,
 1323,
 1348,
 1353,
 1373,
 1382,
 1388,
 1393,
 1418,
 1429,
 1439,
 1597,
 1639,
 1673,
 1725,
 1728,
 1859,
 1886,
 1896,
 1932,
 1960,
 2041,
 2446,
 2604,
 2686,
 3162,
 3242,
 3866,
 3959,
 3963,
 4122,
 4261,
 4434,
 4527,
 4538,
 4539,
 4579,
 4592,
 4649,
 4697,
 4714,
 4729,
 4739,
 4864,
 4902,
 4930,
 5163,
 5171,
 5172,
 5197,
 5272,
 5282,
 5287,
 5294,
 5329,
 5353,
 5528,
 5584,
 562

In [62]:
%time r.SearchRequest('!зимний дворец санкт петербурге гг дворец искусств прошлом главный императорский дворец россии расположенный по адресу дворцовая площадь дворцовая набережная нынешнее здание дворца пятое построено годах русским архитектором итальянского происхождения бартоломео франческо растрелли стиле пышного елизаветинского барокко элементами французского рококо интерьерах начиная советского времени стенах дворца размещена основная экспозиция эрмитажа момента окончания строительства году по год использовался качестве официальной зимней резиденции российских императоров году николай ii перенёс постоянную резиденцию александровский дворец царском селе октября года до ноября года во дворце работал госпиталь имени царевича алексея николаевича июля по ноябрь года во дворце размещалось временное правительство январе года во дворце открыт государственный музей революции разделявший здание государственным эрмитажем вплоть до года зимний дворец дворцовая площадь образуют красивейший архитектурный ансамбль современного города являются одним из главных объектов международного туризма история первый дворец свадебные палаты файл по рисунку махаева вид зимнего дворца jpg thumb right px второй дворец зимний дворец петра третий дворец дворец анны иоанновны всего за период годов городе возводилось пять зимних дворцов первоначально пётр поселился построенном на скорую руку году недалеко от петропавловской крепости одноэтажном доме первый зимний дворец свадебные палаты петра пётр великий владел участком между невой миллионной улицей на месте нынешнего эрмитажного театра году здесь глубине участка строится деревянный зимний дом небольшой двухэтажный дом высоким крыльцом черепичной крышей году были выстроены каменные свадебные палаты петра этот дворец стал подарком губернатора санкт петербурга александра даниловича меншикова свадьбе петра екатерины алексеевны второй зимний дворец дворец петра зимней канавки году архитектор георг маттарнови по приказу царя приступил постройке нового зимнего дворца на углу невы зимней канавки которую тогда называли зимнедомным каналом году пётр со всем своим')

['!зимний', 'дворец', 'санкт', 'петербурге', 'гг', 'дворец', 'искусств', 'прошлом', 'главный', 'императорский', 'дворец', 'россии', 'расположенный', 'по', 'адресу', 'дворцовая', 'площадь', 'дворцовая', 'набережная', 'нынешнее', 'здание', 'дворца', 'пятое', 'построено', 'годах', 'русским', 'архитектором', 'итальянского', 'происхождения', 'бартоломео', 'франческо', 'растрелли', 'стиле', 'пышного', 'елизаветинского', 'барокко', 'элементами', 'французского', 'рококо', 'интерьерах', 'начиная', 'советского', 'времени', 'стенах', 'дворца', 'размещена', 'основная', 'экспозиция', 'эрмитажа', 'момента', 'окончания', 'строительства', 'году', 'по', 'год', 'использовался', 'качестве', 'официальной', 'зимней', 'резиденции', 'российских', 'императоров', 'году', 'николай', 'ii', 'перенёс', 'постоянную', 'резиденцию', 'александровский', 'дворец', 'царском', 'селе', 'октября', 'года', 'до', 'ноября', 'года', 'во', 'дворце', 'работал', 'госпиталь', 'имени', 'царевича', 'алексея', 'николаевича', 'июля', '

set()

In [63]:
%time r.SearchRequest('!зимний дворец санкт петербурге гг дворец искусств прошлом главный императорский дворец россии расположенный по адресу дворцовая площадь дворцовая набережная нынешнее здание дворца пятое построено годах русским архитектором итальянского происхождения бартоломео франческо растрелли стиле пышного елизаветинского барокко элементами французского рококо интерьерах начиная советского времени стенах дворца размещена основная экспозиция эрмитажа момента окончания строительства году по год использовался качестве официальной зимней резиденции российских императоров году николай ii перенёс постоянную резиденцию александровский дворец царском селе октября года до ноября года во дворце работал госпиталь имени царевича алексея николаевича июля по ноябрь года во дворце размещалось временное правительство январе года во дворце открыт государственный музей революции разделявший здание государственным эрмитажем вплоть до года зимний дворец дворцовая площадь образуют красивейший архитектурный ансамбль современного города являются одним из главных объектов международного туризма история первый дворец свадебные палаты файл по рисунку махаева вид зимнего дворца jpg thumb right px второй дворец зимний дворец петра третий дворец дворец анны иоанновны всего за период годов городе возводилось пять зимних дворцов первоначально пётр поселился построенном на скорую руку году недалеко от петропавловской крепости одноэтажном доме первый зимний дворец свадебные палаты петра пётр великий владел участком между невой миллионной улицей на месте нынешнего эрмитажного театра году здесь глубине участка строится деревянный зимний дом небольшой двухэтажный дом высоким крыльцом черепичной крышей году были выстроены каменные свадебные палаты петра этот дворец стал подарком губернатора санкт петербурга александра даниловича меншикова свадьбе петра екатерины алексеевны второй зимний дворец дворец петра зимней канавки году архитектор георг маттарнови по приказу царя приступил ((постройке || нового) && зимнего) дворца на углу невы зимней канавки которую тогда называли зимнедомным каналом году пётр со всем своим')

['!зимний', 'дворец', 'санкт', 'петербурге', 'гг', 'дворец', 'искусств', 'прошлом', 'главный', 'императорский', 'дворец', 'россии', 'расположенный', 'по', 'адресу', 'дворцовая', 'площадь', 'дворцовая', 'набережная', 'нынешнее', 'здание', 'дворца', 'пятое', 'построено', 'годах', 'русским', 'архитектором', 'итальянского', 'происхождения', 'бартоломео', 'франческо', 'растрелли', 'стиле', 'пышного', 'елизаветинского', 'барокко', 'элементами', 'французского', 'рококо', 'интерьерах', 'начиная', 'советского', 'времени', 'стенах', 'дворца', 'размещена', 'основная', 'экспозиция', 'эрмитажа', 'момента', 'окончания', 'строительства', 'году', 'по', 'год', 'использовался', 'качестве', 'официальной', 'зимней', 'резиденции', 'российских', 'императоров', 'году', 'николай', 'ii', 'перенёс', 'постоянную', 'резиденцию', 'александровский', 'дворец', 'царском', 'селе', 'октября', 'года', 'до', 'ноября', 'года', 'во', 'дворце', 'работал', 'госпиталь', 'имени', 'царевича', 'алексея', 'николаевича', 'июля', '

set()

In [64]:
%time r.SearchRequest('руины храма аполлона дельфах файл santuario delfos jpg thumb px модель античного святилища аполлона дельфах де льфы древнегреческий город юго восточной фокиде греция общегреческий религиозный центр храмом оракулом аполлона по легенде был назван по имени сына аполлона дельфа начала vi века до вплоть до конца iv века здесь проходили общегреческие пифийские игры археологический заповедник дельфы включён список всемирного наследия раскопками начавшимися году были открыты храм аполлона пифийского vi iv века до сокровищницы сифносцев около до афинян начало до стоя галерея портик афинян до театр ii век до стадион vi век до другие сооружения честь дельф была названа среда разработки delphi одноимённый язык также извилина дельфы на спутнике юпитера европе география руины древних дельф расположены километрах от побережья коринфского залива город итея на юго западном склоне горы парнаса на высоте метров над уровнем моря современный малый город дельфы находится неподалёку западнее руин община дельфы входит периферийную единицу фокиду община включает себя приморский малый город галаксидион этимология греческое слово восходит корню матка лоно утроба отсюда происходят слова брат или букв единоутробный дельфин новорожденный младенец утробный возможно из за внешнего сходства младенцем или из за того что крик дельфина похож на крик ребёнка причина такого названия видимо связана тем что представлении древних греков неподалёку от храма аполлона находился пуп земли мифология зевс послал краёв света двух орлов они встретились на пифийской скале либо там встретились лебеди либо вороны эта встреча обозначила что там находился пуп земли который охраняли две горгоны некогда доля дельф принадлежала гее она отдала её фемиде та подарила аполлону по трезенскому рассказу дельфы ранее принадлежали посейдону калаврия аполлону позднее они поменялись местностями югу от храма аполлона дельфах был храм геи статую аполлона дельфах виде колонны упоминает евмел городе было также прорицалище диониса по орфикам там гроб старшего диониса он лежит под треножником или омфале см загрей')

['руины', 'храма', 'аполлона', 'дельфах', 'файл', 'santuario', 'delfos', 'jpg', 'thumb', 'px', 'модель', 'античного', 'святилища', 'аполлона', 'дельфах', 'де', 'льфы', 'древнегреческий', 'город', 'юго', 'восточной', 'фокиде', 'греция', 'общегреческий', 'религиозный', 'центр', 'храмом', 'оракулом', 'аполлона', 'по', 'легенде', 'был', 'назван', 'по', 'имени', 'сына', 'аполлона', 'дельфа', 'начала', 'vi', 'века', 'до', 'вплоть', 'до', 'конца', 'iv', 'века', 'здесь', 'проходили', 'общегреческие', 'пифийские', 'игры', 'археологический', 'заповедник', 'дельфы', 'включён', 'список', 'всемирного', 'наследия', 'раскопками', 'начавшимися', 'году', 'были', 'открыты', 'храм', 'аполлона', 'пифийского', 'vi', 'iv', 'века', 'до', 'сокровищницы', 'сифносцев', 'около', 'до', 'афинян', 'начало', 'до', 'стоя', 'галерея', 'портик', 'афинян', 'до', 'театр', 'ii', 'век', 'до', 'стадион', 'vi', 'век', 'до', 'другие', 'сооружения', 'честь', 'дельф', 'была', 'названа', 'среда', 'разработки', 'delphi', 'одноимё

{40}